# Querying Data Lakes with DataFrames

Apache Spark&trade; and Azure Databricks&reg; make it easy to access and work with files stored in Data Lakes, such as Azure Data Lake Storage (ADLS).

Companies frequently store thousands of large data files gathered from various teams and departments, typically using a diverse variety of formats including CSV, JSON, and XML. Data scientists often wish to extract insights from this data.

The classic approach to querying this data is to load it into a central database called a **data warehouse**. Traditionally, data engineers must design the schema for the central database, extract the data from the various data sources, transform the data to fit the warehouse schema, and load it into the central database. A data scientist can then query the data warehouse directly or query smaller data sets created to optimize specific types of queries. The data warehouse approach works well, but requires a great deal of up front effort to design and populate schemas. It also limits historical data, which is constrained to only the data that fits the warehouse’s schema.

An alternative approach is a **Data Lake**, which:

* Is a storage repository that cheaply stores a vast amount of raw data in its native format.
* Consists of current and historical data dumps in various formats including XML, JSON, CSV, Parquet, etc.
* May contain operational relational databases with live transactional data.

Spark is ideal for querying Data Lakes. Spark DataFrames can be used to read directly from raw files contained in a Data Lake and then execute queries to join and aggregate the data.

This lesson illustrates how to perform exploratory data analysis (EDA) to gain insights from a Data Lake.

## Prerequisites
* **IMPORTANT**: You must have permissions within your Azure subscription to create an App Registration and service principal within Azure Active Directory to complete this lesson.
* Lesson: <a href="$./02-Querying-Files">Querying Files with SQL</a>

### Getting Started

Run the following cell to configure our "classroom."

In [3]:
%run "./Includes/Classroom-Setup"

## Create Azure Data Lake Storage Gen1 (ADLS)

1. In the [Azure portal](https://portal.azure.com), select **+ Create a resource**, enter "data lake" into the Search the Marketplace box, select **Data Lake Storage Gen1** from the results, and then select **Create**.

   ![In the Azure portal, +Create a resource is highlighted in the navigation pane, "data lake" is entered into the Search the Marketplace box, and Data Lake Storage Gen1 is highlighted in the results.](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-create-resource.png 'Create Azure Data Lake Storage Gen1')

2. On the New Data Lake Storage Gen1 blade, enter the following:

   - **Name**: Enter a globally unique name (indicated by a green check mark).
   - **Subscription**: Select the subscription you are using for this module.
   - **Resource group**: Choose your module resource group.
   - **Location**: Select the closest location.
   - **Pricing package**: Choose Pay-as-You-Go.
   - **Encryption settings**: Leave set to the default value of Enabled.

   ![The New Data Lake Storage Gen1 blade is displayed, with the previously mentioned settings entered into the appropriate fields.](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-create-new.png 'New Data Lake Storage Gen1')

3. Select **Create** to provision the new ADLS instance.

4. In the cell below, set the value of the `adlsAccountName` variable to the same name you used for the **Name** field when creating your ADLS instance above, and then run the cell.

In [5]:
adlsAccountName = "<your-data-lake-store-account-name>"

## Create Azure Active Directory application and service principal

> **IMPORTANT**: You must have permissions within your Azure subscription to create an App registration and service principal within Azure Active Directory to complete this lesson.

ADLS uses Azure Active Directory for authentication. To provide access to your ADLS instance from Azure Databricks, you will use [service-to-service authentication](https://docs.microsoft.com/en-us/azure/data-lake-store/data-lake-store-service-to-service-authenticate-using-active-directory). For this, you need to create an identity in Azure Active Directory (Azure AD) known as a service principal.

1. In the [Azure portal](https://portal.azure.com), select **Azure Active Directory** from the left-hand navigation menu, select **App registrations**, and then select **+ New application registration**.

   ![Register new app in Azure Active Directory](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-app-registration.png 'Register new app in Azure Active Directory')

2. On the Create blade, enter the following:

  * **Name**: Enter a unique name, such as databricks-demo (this name must be unique, as indicated by a green check mark).
  * **Application type**: Select Web app / API.
  * **Sign-on URL**: Enter https://databricks-demo.com.

   ![Create a new app registration](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-app-create.png 'Create a new app registration')

3. Select **Create**.

4. To access your ADLS instance from Azure Databricks you will need to provide the credentials of your newly created service principal within Databricks. On the Registered app blade that appears, copy the **Application ID** and paste it into the cell below as the value for the `clientId` variable.

   ![Copy the Registered App Application ID](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-id.png 'Copy the Registered App Application ID')

5. Next, select **Settings** on the Registered app blade, and then select **Keys**.

   ![Open Keys blade for the Registered App](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-settings-keys.png 'Open Keys blade for the Registered App')

6. On the Keys blade, you will create a new password by doing the following under Passwords:

  * **Description**: Enter a description, such as ADLS Auth.
  * **Expires**: Select a duration, such as In 1 year.

  ![Create new password](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-create-key.png 'Create new password')

7. Select **Save**, and then copy the key displayed under **Value**, and paste it into the cell below for the value of the `clientKey` variable. **Note**: This value will not be accessible once you navigate away from this screen, so make sure you copy it before leaving the Keys blade.

  ![Copy key value](https://databricksdemostore.blob.core.windows.net/images/04/06/registered-app-key-value.png 'Copy key value')

8. Run the cell below.

In [7]:
clientId = "<your-service-client-id>"
clientKey = "<your-service-credentials>"

## Retrieve your Azure AD tenant ID

To perform authentication using the service principal account, Databricks uses OAUTH2. For this, you need to provide your Azure AD Tenant ID.

1. To retrieve your tenant ID, select **Azure Active Directory** from the left-hand navigation menu in the Azure portal, then select **Properties**, and select the copy button next to **Directory ID** on the Directory Properties blade.

   ![Retrieve Tenant ID](https://databricksdemostore.blob.core.windows.net/images/04/06/aad-tenant-id.png 'Retrieve Tenant ID')

2. Paste the copied value into the cell below for the value of the `tenantId` variable, and then run the cell.

In [9]:
tenantId = "<your-directory-id>"

## Assign permissions to the service principal in ADLS

Next, you need to assign the required permissions to the service principal in ADLS.

1. In the [Azure portal](https://portal.azure.com), navigate to the ADLS instance you created above, and on the Overview blade, select **Data explorer**.

   ![ADLS Overview blade](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-overview.png 'ADLS Overview blade')

2. In the Data Explorer blade, select **Access** on the toolbar.

   ![ADLS Data Explorer toolbar](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-data-explorer-toolbar.png 'ADLS Data Explorer toolbar')

3. On the Access blade, select **+ Add**.

   ![ADLS Data Explorer add access](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-access.png 'ADLS Data Explorer add access')

4. On the Assign permissions -> Select user or group blade, enter the name of your Registered app (e.g., databricks-demo) into the **Select** box, choose your app from the list, and select **Select**.

   ![ADLS assign permissions to user or group](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assign-permissions-select-user-or-group.png 'ADLS assign permissions to user or group')

5. On the Assign permissions -> Select permissions blade, set the following:

  * **Permissions**: Check **Read**, **Write**, and **Execute**.
  * **Add to**: Choose This folder and all children.
  * **Add as**: Choose An access permission entry.
  
  ![ADLS assign permissions](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assign-permissions.png 'ADLS assign permissions')

6. Select **Ok**

7. You will now see the service principal listed under **Assigned permissions** on the Access blade.

  ![ADLS assigned permissions](https://databricksdemostore.blob.core.windows.net/images/04/06/adls-assigned-permissions.png 'ADLS assigned permissions')

## Mount ADLS to DBFS

You are now ready to access your ADLS account from Azure Databricks. Run the cell below to set the required configuration and mount ADLS to DBFS.

In [12]:
configs = {"dfs.adls.oauth2.access.token.provider.type": "ClientCredential",
           "dfs.adls.oauth2.client.id": clientId,
           "dfs.adls.oauth2.credential": clientKey,
           "dfs.adls.oauth2.refresh.url": "https://login.microsoftonline.com/" + tenantId + "/oauth2/token"}

dbutils.fs.mount(
  source = "adl://" + adlsAccountName + ".azuredatalakestore.net/",
  mount_point = "/mnt/adls",
  extra_configs = configs)

## Copy data to ADLS

Run the following cell to copy the Crime-data-2016 dataset from the Training folder into your ADLS instance, in a folder named "training". This will take a few minutes to complete.

In [14]:
dbutils.fs.cp("/mnt/training/crime-data-2016", "mnt/adls/training/crime-data-2016", true)

## Looking at the files in our Data Lake

Start by reviewing which files are in our Data Lake.

In `dbfs:/mnt/adls/training/crime-data-2016`, there are Parquet files containing 2016 crime data from several United States cities.

In the cell below we have data for Boston, Chicago, New Orleans, and more.

In [16]:
%fs ls /mnt/adls/training/crime-data-2016

The next step in looking at the data is to create a DataFrame for each file.  

Start by creating a DataFrame of the data from New York and then Boston:

| City          | Table Name              | Path to DBFS file
| ------------- | ----------------------- | -----------------
| **New York**  | `CrimeDataNewYork`      | `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-New-York-2016.parquet`
| **Boston**    | `CrimeDataBoston`       | `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-Boston-2016.parquet`

In [18]:
crimeDataNewYorkDF = spark.read.parquet("/mnt/adls/training/crime-data-2016/Crime-Data-New-York-2016.parquet")

In [19]:
crimeDataBostonDF = spark.read.parquet("/mnt/adls/training/crime-data-2016/Crime-Data-Boston-2016.parquet")

With the two DataFrames created, it is now possible to review the first couple records of each file.

Notice in the example below:
* The `crimeDataNewYorkDF` and `crimeDataBostonDF` DataFrames use different names for the columns.
* The data itself is formatted differently and different names are used for similar concepts.

This is common in a Data Lake. Often files are added to a Data Lake by different groups at different times. The advantage of this strategy is that anyone can contribute information to the Data Lake and that Data Lakes scale to store arbitrarily large and diverse data. The tradeoff for this ease in storing data is that it doesn’t have the rigid structure of a traditional relational data model, so the person querying the Data Lake will need to normalize data before extracting useful insights.

The alternative to a Data Lake is a data warehouse.  In a data warehouse, a committee often regulates the schema and ensures data is normalized before being made available.  This makes querying much easier but also makes gathering the data much more expensive and time-consuming.  Many companies choose to start with a Data Lake to accumulate data.  Then, as the need arises, they normalize data and produce higher quality tables for querying.  This reduces the up front costs while still making data easier to query over time.  The normalized tables can later be loaded into a formal data warehouse through nightly batch jobs.  In this way, Apache Spark is used to manage and query both Data Lakes and data warehouses.

In [21]:
display(crimeDataNewYorkDF)

In [22]:
display(crimeDataBostonDF)

## Same Type of Data, Different Structure

In this section, we examine crime data to determine how to extract homicide statistics.

Because the data sets are pooled together in a Data Lake, each city may use different field names and values to indicate homicides, dates, etc.

For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or "MURDER".
* In the New York data, the column is named `offenseDescription` while in the Boston data, the column is named `OFFENSE_CODE_GROUP`.
* In the New York data, the date of the event is in the `reportDate`, while in the Boston data, there is a single column named `MONTH`.

To get started, create a temporary view containing only the homicide-related rows.

At the same time, normalize the data structure of each table so all the columns (and their values) line up with each other.

In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |

For the upcoming aggregation, you need to alter the New York data set to include a `month` column which can be computed from the `reportDate` column using the `month()` function. Boston already has this column.

In this example, we use several functions in the `pyspark.sql.functions` library, and need to import:

* `month()` to extract the month from `reportDate` timestamp data type.
* `lower()` to convert text to lowercase.
* `contains(mySubstr)` to indicate a string contains substring `mySubstr`.

Also, note we use  `|`  to indicate a logical `or` of two conditions in the `filter` method.

In [25]:
from pyspark.sql.functions import lower, upper, month, col

homicidesNewYorkDF = (crimeDataNewYorkDF 
  .select(month(col("reportDate")).alias("month"), col("offenseDescription").alias("offense")) 
  .filter(lower(col("offenseDescription")).contains("murder") | lower(col("offenseDescription")).contains("homicide"))
)

display(homicidesNewYorkDF)

Notice how the same kind of information is presented differently in the Boston data:

`offense` is called `OFFENSE_CODE_GROUP` and there is only one category `homicide`.

In [27]:
homicidesBostonDF = (crimeDataBostonDF 
  .select("month", col("OFFENSE_CODE_GROUP").alias("offense")) 
  .filter(lower(col("OFFENSE_CODE_GROUP")).contains("homicide"))
)

display(homicidesBostonDF)

See below the structure of the two tables is now identical.

In [29]:
display(homicidesNewYorkDF.limit(5))

In [30]:
display(homicidesBostonDF.limit(5))

## Analyzing the Data

-sandbox
Now that you normalized the homicide data for each city, combine the two by taking their union.

When done, aggregate that data to compute the number of homicides per month.

Start by creating a new DataFrame called `homicidesBostonAndNewYorkDF` that consists of the `union` of `homicidesNewYorkDF` with `homicidesBostonDF`.

In [33]:
homicidesBostonAndNewYorkDF = homicidesNewYorkDF.union(homicidesBostonDF)

See all the data in one table below:

In [35]:
display(homicidesBostonAndNewYorkDF.orderBy("month"))

And finally, perform a simple aggregation to see the number of homicides per month:

In [37]:
display(homicidesBostonAndNewYorkDF.select("month").orderBy("month").groupBy("month").count())

## Exercise 1

Merge the crime data for Chicago with the data for New York and Boston, and then update our final aggregation of counts-by-month.

### Step 1

Create the initial DataFrame of the Chicago data.
0. The source file is `dbfs:/mnt/adls/training/crime-data-2016/Crime-Data-Chicago-2016.parquet`.
0. Name the view `crimeDataChicagoDF`.
0. View the data by invoking the `show()` method.

In [40]:
# ANSWER
crimeDataChicagoDF = spark.read.parquet("/mnt/adls/training/crime-data-2016/Crime-Data-Chicago-2016.parquet")
display(crimeDataChicagoDF)

In [41]:
# TEST - Run this cell to test your solution.

total = crimeDataChicagoDF.count()

dbTest("DF-L6-crimeDataChicago-count", 267872, total)

print("Tests passed!")

-sandbox
### Step 2

Create a new view that normalizes the data structure.
0. Name the DataFrame `homicidesChicagoDF`.
0. The DataFrame should have at least two columns: `month` and `offense`.
0. Filter the data to include only homicides.
0. View the data by invoking the `show()` method.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Use the `month()` function to extract the month-of-the-year.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To find out which values for each offense constitutes a homicide, produce a distinct list of values from the`crimeDataChicagoDF` DataFrame.

In [43]:
# ANSWER

display(crimeDataChicagoDF.select("primaryType").distinct().orderBy("primaryType"))

In [44]:
# ANSWER

homicidesChicagoDF = (crimeDataChicagoDF 
  .select(month(col("date")).alias("month"), col("primaryType").alias("offense")) 
  .filter(lower(col("primaryType")) == "homicide")
)

display(homicidesChicagoDF)

In [45]:
# TEST - Run this cell to test your solution.
homicidesChicago = homicidesChicagoDF.select("month").groupBy("month").count().orderBy("month").collect()

dbTest("DF-L6-homicideChicago-len", 12, len(homicidesChicago))
dbTest("DF-L6-homicideChicago-0", 54, homicidesChicago[0][1])
dbTest("DF-L6-homicideChicago-6", 71, homicidesChicago[6][1])
dbTest("DF-L6-homicideChicago-11", 58, homicidesChicago[11][1])

print("Tests passed!")

-sandbox
### Step 3

Create a new DataFrame that merges all three data sets (New York, Boston, Chicago):
0. Name the view `allHomicidesDF`.
0. Use the `union()` method introduced earlier to merge all three tables.
  * `homicidesNewYorkDF`
  * `homicidesBostonDF`
  * `homicidesChicagoDF`
0. View the data by invoking the `show()` method.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** To union three tables together, copy the previous example and apply a `union()` method again.

In [47]:
# ANSWER
allHomicidesDF = homicidesNewYorkDF.union(homicidesBostonDF).union(homicidesChicagoDF)
display(allHomicidesDF)

In [48]:
# TEST - Run this cell to test your solution.

allHomicides = allHomicidesDF.count()
dbTest("DF-L6-allHomicides-count", 1203, allHomicides)

print("Tests passed!")

### Step 4

Create a new DataFrame that counts the number of homicides per month.
0. Name the DataFrame `homicidesByMonthDF`.
0. Rename the column `count(1)` to `homicides`.
0. Group the data by `month`.
0. Sort the data by `month`.
0. Count the number of records for each aggregate.
0. View the data by invoking the `show()` method.

In [50]:
# ANSWER
from pyspark.sql.functions import count
homicidesByMonthDF = allHomicidesDF.select("month").groupBy("month").agg(count('*').alias("homicides")).orderBy("month")
display(homicidesByMonthDF)

In [51]:
# TEST - Run this cell to test your solution.
allHomicides = homicidesByMonthDF.collect()

dbTest("DF-L6-homicidesByMonth-len", 12, len(allHomicides))
dbTest("DF-L6-homicidesByMonth-0", 1, allHomicides[0][0])
dbTest("DF-L6-homicidesByMonth-11", 12, allHomicides[11][0])
dbTest("DF-L6-allHomicides-0", 83, allHomicides[0][1])
dbTest("DF-L6-allHomicides-1", 83, allHomicides[0][1])
dbTest("DF-L6-allHomicides-2", 68, allHomicides[1][1])
dbTest("DF-L6-allHomicides-3", 72, allHomicides[2][1])
dbTest("DF-L6-allHomicides-4", 76, allHomicides[3][1])
dbTest("DF-L6-allHomicides-5", 105, allHomicides[4][1])
dbTest("DF-L6-allHomicides-6", 120, allHomicides[5][1])
dbTest("DF-L6-allHomicides-7", 116, allHomicides[6][1])
dbTest("DF-L6-allHomicides-8", 144, allHomicides[7][1])
dbTest("DF-L6-allHomicides-9", 109, allHomicides[8][1])
dbTest("DF-L6-allHomicides-10", 109, allHomicides[9][1])
dbTest("DF-L6-allHomicides-11", 111, allHomicides[10][1])
dbTest("DF-L6-allHomicides-12", 90, allHomicides[11][1])

print("Tests passed!")

## Unmount ADLS from DBFS

In [53]:
dbutils.fs.unmount("/mnt/adls")

## Summary

* Spark DataFrames allow you to easily manipulate data in a Data Lake.

## Review Questions
**Q:** What is a Data Lake?  
**A:** A Data Lake is a collection of data files gathered from various sources.  Spark loads each file as a table and then executes queries by joining and aggregating these files.

**Q:** What are advantages of Data Lakes over classic Data Warehouses?  
**A:** Data Lakes allow for large amounts of data to be aggregated from many sources with minimal preparatory steps.  Data Lakes also allow for very large files.  Powerful query engines such as Spark can read the diverse collection of files and execute complex queries efficiently.

**Q:** What are some advantages of Data Warehouses?  
**A:** Data warehouses are neatly curated to ensure data from all sources fit a common schema.  This makes them easy to query.

**Q:** What's the best way to combine the advantages of Data Lakes and Data Warehouses?  
**A:** Start with a Data Lake.  As you query, you will discover cases where the data needs to be cleaned, combined, and made more accessible.  Create periodic Spark jobs to read these raw sources and write new "golden" DataFrames that are cleaned and more easily queried.